In [25]:
import ee
import time
import sys
import numpy as np
import pandas as pd
import itertools
import os
import urllib
import geopandas as gpd
import math

In [2]:
import rasterio
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

from pprint import pprint
from rasterio.features import Window
from rasterio.windows import bounds
from shapely.geometry import MultiPolygon, box
from PIL import Image
from rasterio.features import Window
from subprocess import call
from IPython import display


In [7]:
test_tif = 'data/1_1.tif'

In [8]:
satdat = rasterio.open(test_tif)
img = satdat.read()

In [9]:
# dataset name
print(satdat.name)
# number of bands in this dataset
print(satdat.count)
# shape of one band
print(satdat.shape)
# The dataset shape
print(img.shape)
# And provides a sequence of band indexes.  These are one indexing, not zero indexing like Numpy arrays.
#print(satdat.indexes)

# pix range
print(np.max(img),np.min(img))
# other profile info
p = satdat.profile.copy()
print(p)

data/1_1.tif
3087
(97, 160)
(3087, 97, 160)
nan nan
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 160, 'height': 97, 'count': 3087, 'crs': CRS.from_wkt('PROJCS["MODIS Sinusoidal",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Sinusoidal"],PARAMETER["longitude_of_center",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': Affine(463.3127165279165, 0.0, -8158936.938050615,
       0.0, -463.3127165279167, 3637004.8247441463), 'blockxsize': 256, 'blockysize': 256, 'tiled': True, 'compress': 'lzw', 'interleave': 'pixel'}


ERROR 1: PROJ: proj_identify: Cannot find proj.db
ERROR 1: PROJ: proj_as_wkt: Cannot find proj.db


In [11]:
satdat.crs

ERROR 1: PROJ: proj_identify: Cannot find proj.db


CRS.from_wkt('PROJCS["MODIS Sinusoidal",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Sinusoidal"],PARAMETER["longitude_of_center",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]')

In [12]:
# Convert pixel coordinates to world coordinates.
# Upper left pixel
row_min = 0
col_min = 0

# Lower right pixel.  Rows and columns are zero indexing.
row_max = satdat.height - 1
col_max = satdat.width - 1

# Transform coordinates with the dataset's affine transformation.
topleft = satdat.transform * (row_min, col_min)
botright = satdat.transform * (row_max, col_max)

print("Top left corner coordinates: {}".format(topleft))
print("Bottom right corner coordinates: {}".format(botright))

Top left corner coordinates: (-8158936.938050615, 3637004.8247441463)
Bottom right corner coordinates: (-8114458.917263934, 3563338.1028162073)


In [13]:
satdat.bounds

BoundingBox(left=-8158936.938050615, bottom=3592063.491240938, right=-8084806.903406148, top=3637004.8247441463)

In [14]:
from pyproj import CRS, Transformer

In [15]:
latlog = CRS.from_proj4("+proj=latlon")

In [16]:
latlog

<Geographic 2D CRS: +proj=latlon +type=crs>
Name: unknown
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [17]:
transformer = Transformer.from_crs(satdat.crs, latlog)

ERROR 1: PROJ: proj_as_wkt: Cannot find proj.db


In [18]:
lon, lat = transformer.transform(-8310440.196355244, 3530906.2126592537)
print(lat,lon)

31.901283959078533 -87.85355239014226


In [19]:
print(img.shape)

(3087, 97, 160)


In [22]:
img2= np.transpose(img,axes=(1,2,0))
print(img2.shape)

(97, 160, 3087)


In [23]:
bands = 7 
composite_period = 8 


In [26]:
bands_per_year = bands * math.ceil(365 / composite_period)

In [27]:
bands_per_year

322

In [28]:
322/7

46.0

In [29]:
total_bands = img2.shape[2]

In [30]:
total_bands

3087

In [43]:
image_list = []
for i in range(0,total_bands-7,bands_per_year):
    #print(i,i+bands_per_year)
    image_list.append(img2[:, :, i : i + bands_per_year])

In [44]:
len(image_list)

10

In [49]:
image_list[8].shape

(97, 160, 322)

In [ ]:
image_list = []
cur_idx = 0
for i in range(0, num_years - 1):
    image_list.append(img[:, :, cur_idx : cur_idx + bands_per_year])
    cur_idx += bands_per_year
image_list.append(img[:, :, cur_idx:])
return image_list